Explicación del Código:

Carga de los datos: Los datos iniciales se cargan en un DataFrame de pandas, con las columnas que mencionaste, incluyendo arrival_delay_minutes.
Función assign_delay_cause:

Si arrival_delay_minutes es menor o igual a 5, la causa del retraso es "Ninguna".

Si arrival_delay_minutes es mayor que 5 pero menor o igual a 60, seleccionamos una causa aleatoria de un conjunto de opciones, asegurándonos de que las opciones "Clima origen" y "Clima destino" se asignen con una probabilidad 10 veces menor.

Si arrival_delay_minutes es mayor a 60, la causa es exclusivamente "Mantenimiento".

Aplicación de la función: La función assign_delay_cause se aplica a cada fila del DataFrame usando apply.

Mostrar el resultado: El DataFrame resultante se imprime, mostrando solo las columnas relevantes (flight_id, flight_no, arrival_delay_minutes, delay_cause).

In [6]:
import pandas as pd
import random

#Se guarda ruta del archivo en la cadena archivo_latam

flights = r'C:\Users\flore\OneDrive\Documents\Proyecto Final Clonado\ProyectoIntegradorDAFT08\Data\Data Clean\flightdf_data_clean.csv'


# Leer el archivo CSV

df = pd.read_csv(flights)


# Función para asignar la causa del retraso
def assign_delay_cause(row):
    if row['arrival_delay_minutes'] <= 5:
        return 'No delays'
    elif row['arrival_delay_minutes'] > 5 and row['arrival_delay_minutes'] <= 60:
        # Opciones posibles para retrasos mayores a 5 pero menores o iguales a 60
        options = ['carrier_ct', 'late_aircraft_ct', 'nas_ct', 'security_ct', 
                   'weather_ct']
        # Asegurarse de que las opciones "Clima origen" y "Clima destino" ocurran 10 veces menos
        if random.random() < 0.1:
            return random.choice([opt for opt in options if 'weather_ct' in opt])  # Selección de opciones Clima
        else:
            return random.choice([opt for opt in options if 'weather_ct' not in opt])  # Otras opciones
    else:
        return 'carrier_ct'  # Si el retraso es mayor a 60 minutos, solo puede ser Mantenimiento

# Aplicamos la función para asignar la causa del retraso a cada fila
df['delay_cause'] = df.apply(assign_delay_cause, axis=1)

# Clasificar los retrasos
def classify_delay(minutes):
    if minutes <= 5:
        return 'On-time'
    elif minutes <= 30:
        return 'Short delay'
    elif minutes <= 60:
        return 'Moderate delay'
    else:
        return 'Long delay'

df['delay_classification'] = df['arrival_delay_minutes'].apply(classify_delay)

# Añadir columna binaria para vuelos puntuales
df['on_time'] = df['arrival_delay_minutes'] <= 5

# Mostrar las primeras filas con las nuevas columnas
print(df[['flight_id', 'flight_no', 'arrival_delay_minutes', 'delay_cause', 'delay_classification', 'on_time']])

# Contar frecuencias de las causas de retraso
cause_counts = df['delay_cause'].value_counts()
print(cause_counts)


       flight_id flight_no  arrival_delay_minutes  delay_cause  \
0           1185    PG0134                   35.0   weather_ct   
1           3979    PG0052                   23.0   weather_ct   
2           4739    PG0561                   11.0  security_ct   
3           5502    PG0529                   32.0   carrier_ct   
4           6938    PG0461                   24.0       nas_ct   
...          ...       ...                    ...          ...   
33116      33117    PG0063                    0.0    No delays   
33117      33118    PG0063                    5.0    No delays   
33118      33119    PG0063                   -1.0    No delays   
33119      33120    PG0063                    2.0    No delays   
33120      33121    PG0063                   -1.0    No delays   

      delay_classification  on_time  
0           Moderate delay    False  
1              Short delay    False  
2              Short delay    False  
3           Moderate delay    False  
4              Sh

In [7]:
df.columns

Index(['flight_id', 'flight_no', 'scheduled_departure', 'scheduled_arrival',
       'departure_airport', 'arrival_airport', 'status', 'aircraft_code',
       'actual_departure', 'actual_arrival', 'departure_delay_minutes',
       'arrival_delay_minutes', 'duration_mm', 'duration', 'duration_hh_mm',
       'delay_cause', 'delay_classification', 'on_time'],
      dtype='object')

In [8]:
# Select the desired columns
selected_columns = ['flight_id','flight_no', 'delay_cause', 'delay_classification', 'on_time']
df = df[selected_columns]

# Print the DataFrame
print(df)

       flight_id flight_no  delay_cause delay_classification  on_time
0           1185    PG0134   weather_ct       Moderate delay    False
1           3979    PG0052   weather_ct          Short delay    False
2           4739    PG0561  security_ct          Short delay    False
3           5502    PG0529   carrier_ct       Moderate delay    False
4           6938    PG0461       nas_ct          Short delay    False
...          ...       ...          ...                  ...      ...
33116      33117    PG0063    No delays              On-time     True
33117      33118    PG0063    No delays              On-time     True
33118      33119    PG0063    No delays              On-time     True
33119      33120    PG0063    No delays              On-time     True
33120      33121    PG0063    No delays              On-time     True

[33121 rows x 5 columns]


In [9]:
df.to_csv("Causes of delays.csv", index=False)
